In [4]:
from datetime import datetime
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.utils import compute_class_weight
from tensorflow import keras
from src.data.load_data import train_X, train_y, validation_X, validation_y, test_X, test_y
from src.models.model import NextItemPredictor
from src.models.predict import predict, avg_precision
from src.models.train import train
import numpy as np
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [2]:
logdir = "../logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [6]:
%tensorboard --logdir ../logs/

In [ ]:
model = NextItemPredictor()
model.compile()

In [4]:
class_weight = compute_class_weight('balanced', np.unique(train_y), train_y)


In [ ]:
train(
    model,
    train_X=train_X,
    train_y=train_y,
    validation_X=validation_X,
    validation_y=validation_y,
    class_weight=class_weight,
    batch_size=8,
    epochs=30,
    callbacks=[tensorboard_callback],
    description='batch100-epochs30'
)

Train on 4081 samples, validate on 1575 samples
Epoch 1/30
4081/4081 [==============================] - 101s 25ms/sample - loss: 7.8641 - val_loss: 7.6836
Epoch 2/30
4081/4081 [==============================] - 95s 23ms/sample - loss: 7.1879 - val_loss: 7.7839
Epoch 3/30
4081/4081 [==============================] - 95s 23ms/sample - loss: 6.9732 - val_loss: 7.9943
Epoch 4/30
4081/4081 [==============================] - 97s 24ms/sample - loss: 6.8018 - val_loss: 8.1753
Epoch 5/30
4081/4081 [==============================] - 91s 22ms/sample - loss: 6.6878 - val_loss: 8.1830
Epoch 6/30
4081/4081 [==============================] - 97s 24ms/sample - loss: 6.5898 - val_loss: 8.3248
Epoch 7/30
4081/4081 [==============================] - 101s 25ms/sample - loss: 6.4878 - val_loss: 8.3596
Epoch 8/30
4081/4081 [==============================] - 96s 24ms/sample - loss: 6.3840 - val_loss: 8.4444
Epoch 9/30
4081/4081 [==============================] - 129s 32ms/sample - loss: 6.2567 - val_loss: 8.

In [7]:
y_pred = predict(model, test_X)

In [8]:
avg_precision(test_y, y_pred)

/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


5.865721894393543e-06

In [9]:
recall_score(test_y, y_pred, average='weighted')

/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.00563063063063063

In [9]:
y_pred = predict(model, train_X)

In [10]:
avg_precision(train_y, y_pred)

/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.005560208949059347

In [11]:
recall_score(train_y, y_pred, average='weighted')


/home/viktoria/Documents/FIIT/NSIETE/NSIETE-project/venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.018132810585640775

In [ ]:
# TODO: bidirectional, recommend 10 items, try to split by users?

# attention, hierarchy